In [ ]:
# Célula 1 — preparação do ambiente e fórmula analítica (exemplo)
import numpy as np
import pandas as pd
import os
# detectar Colab
IN_COLAB = False
try:
    import google.colab
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    print('Executando no Colab — se desejar salvar modelos monte o Drive: from google.colab import drive; drive.mount(...)')

# Parâmetros de exemplo para demonstração
s_0 = 0.0
m = 5.0  # kg
k = 0.5  # coeficiente de arrasto (modelo linear usado aqui)
t = 5.0  # segundos
g = 9.81  # aceleração da gravidade (m/s^2)

# Fórmulas analíticas (modelo com arrasto linear)
v_analitica = (m * g / k) * (1 - np.exp(-k * t / m))
y_analitica = (m * g / k) * t - (m**2 * g / k**2) * (1 - np.exp(-k * t / m))

print(f'Velocidade analítica: {v_analitica:.4f} m/s')
print(f'Posição analítica: {y_analitica:.4f} m')

Velocidade analítica: 38.60 m/s
Posição analítica: 104.51 m


In [ ]:
# Célula 2 — gerar dataset de exemplos (modelo analítico com arrasto linear)
# Definindo constantes
g = 9.81  # aceleração da gravidade (m/s^2)

# Geração de dados
np.random.seed(42)
n_samples = 1000

massas = np.random.uniform(1.0, 10.0, n_samples)  # massa entre 1 e 10 kg
k_arrastos = np.random.uniform(0.1, 1.0, n_samples)  # constante de arrasto entre 0.1 e 1
tempos = np.random.uniform(0.0, 10.0, n_samples)  # tempo entre 0 e 10 segundos

velocidades = []
posicoes = []

for m_val, k_val, t_val in zip(massas, k_arrastos, tempos):
    v_t = (m_val * g / k_val) * (1 - np.exp(-k_val * t_val / m_val))
    y_t = (m_val * g / k_val) * t_val - (m_val**2 * g / k_val**2) * (1 - np.exp(-k_val * t_val / m_val))
    velocidades.append(v_t)
    posicoes.append(y_t)

# Criação do DataFrame
df = pd.DataFrame({
    'massa': massas,
    'constante_de_arrasto': k_arrastos,
    'tempo': tempos,
    'velocidade': velocidades,
    'posicao': posicoes
})
# Salvar em um arquivo CSV local
os.makedirs('data', exist_ok=True)
csv_path = 'data/queda_objeto.csv'
df.to_csv(csv_path, index=False)
print('Dataset salvo em', csv_path)
df.head()

ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

# Carregar os dados do CSV (caminho atualizado)
csv_path = 'data/queda_objeto.csv'
if not os.path.exists(csv_path):
    raise SystemExit(f'Arquivo de dados não encontrado: {csv_path} - execute a célula de geração de dados primeiro')
df = pd.read_csv(csv_path)

# Separar as entradas (X) e as saídas (y)
X = df[['massa', 'constante_de_arrasto', 'tempo']]
y = df[['velocidade', 'posicao']]

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizar os dados
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# salvar scalers para inferência posterior
os.makedirs('model', exist_ok=True)
import joblib
joblib.dump(scaler_X, 'model/scaler_X.pkl')
joblib.dump(scaler_y, 'model/scaler_y.pkl')
print('Scalers salvos em model/')

In [ ]:
# Construir a rede neural
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(2, activation='linear')  # Saída de 2 dimensões: velocidade e posição
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

c:\GitHub\ProjetosPessoais\redeNeuralArtificial\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Treinar o modelo com callbacks (salvar melhor modelo)
checkpoint_cb = ModelCheckpoint('model/queda_objeto_best.keras', save_best_only=True, monitor='val_loss')
earlystop_cb = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[checkpoint_cb, earlystop_cb])

# salvar o modelo final
model.save('model/queda_objeto.keras')
print('Modelo salvo em model/queda_objeto.keras')

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.7815 - mae: 0.7172 - val_loss: 0.3638 - val_mae: 0.4854
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2932 - mae: 0.4107 - val_loss: 0.0971 - val_mae: 0.2246
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0843 - mae: 0.2095 - val_loss: 0.0482 - val_mae: 0.1629
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0320 - mae: 0.1386 - val_loss: 0.0299 - val_mae: 0.1255
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0235 - mae: 0.1178 - val_loss: 0.0234 - val_mae: 0.1086
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0166 - mae: 0.0992 - val_loss: 0.0196 - val_mae: 0.0972
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0132 - mae: 0.0858 - val_loss: 0.0175 - val_mae: 0.0903
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0122 - mae: 0.0823 - val_loss: 0.0151 - val_mae: 0.0849
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.00

In [ ]:
# Avaliar o modelo (carregar melhor modelo salvo se existir)
import os
best_path = 'model/queda_objeto_best.keras'
if os.path.exists(best_path):
    from tensorflow.keras.models import load_model
    model = load_model(best_path)
    print('Carregado melhor modelo salvo em', best_path)

loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae:.6f}')

# Modelo final já salvo na etapa de treino como model/queda_objeto.keras

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011 - mae: 0.0212  
Mean Absolute Error on test data: 0.021667804569005966


In [ ]:
# Célula de inferência — carregar modelo e scalers e testar uma entrada
import numpy as np
import os
from tensorflow.keras.models import load_model
import joblib

model_path = 'model/queda_objeto.keras'
if not os.path.exists(model_path):
    raise SystemExit('Modelo não encontrado: ' + model_path + ' - execute a célula de treino primeiro')
model = load_model(model_path)
scaler_X = joblib.load('model/scaler_X.pkl')
scaler_y = joblib.load('model/scaler_y.pkl')

# Exemplo de entrada e previsão
m_val = 5.0
k_val = 0.5
t_val = 5.0
entrada = np.array([[m_val, k_val, t_val]])
entrada_pad = scaler_X.transform(entrada)
saida_pad = model.predict(entrada_pad)
saida = scaler_y.inverse_transform(saida_pad)
v_pred, y_pred = saida[0]
print(f'Previsão da rede — velocidade: {v_pred:.4f} m/s, posição: {y_pred:.4f} m')

# comparar com solução analítica (mesmo modelo linear usado para gerar dados)
v_analitica = (m_val * 9.81 / k_val) * (1 - np.exp(-k_val * t_val / m_val))
y_analitica = (m_val * 9.81 / k_val) * t_val - (m_val**2 * 9.81 / k_val**2) * (1 - np.exp(-k_val * t_val / m_val))
print(f'Analítica — velocidade: {v_analitica:.4f} m/s, posição: {y_analitica:.4f} m')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Resultados Formula: Velocidade = 38.5993 m/s, Posição = 104.5066 m
Previsões da Rede Neural: Velocidade = 38.4627 m/s, Posição = 102.8974 m


c:\GitHub\ProjetosPessoais\redeNeuralArtificial\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
